In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn import decomposition
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import time

In [0]:
pip install git+https://github.com/samueljackson92/coranking.git

  Cloning https://github.com/samueljackson92/coranking.git to /tmp/pip-req-build-suzmarpn
  Running command git clone -q https://github.com/samueljackson92/coranking.git /tmp/pip-req-build-suzmarpn
  Created wheel for coranking: filename=coranking-0.1.1-cp36-cp36m-linux_x86_64.whl size=87038 sha256=ed51a27bed1d6e06d22f0866ff21dc20442239aff78671f8a8509fff7be9ef50
  Stored in directory: /tmp/pip-ephem-wheel-cache-ahsp9a9p/wheels/8e/e0/11/4ae911b6fc472b8bf6087e29d861d2b798c119603adf4b93fa
Successfully built coranking


In [0]:
import coranking
from coranking.metrics import trustworthiness, continuity, LCMC


###1. Loading and Processing MNIST

In [0]:
link1='https://drive.google.com/open?id=1LAXOvFLyMmK8Pkdw6efWhGHtIQpkGnYd'

fluff, id = link1.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv') 
training_data  = pd.read_csv('train.csv')

1LAXOvFLyMmK8Pkdw6efWhGHtIQpkGnYd


In [0]:
labels  = training_data['label']
training_data = training_data.drop("label",axis=1)

In [0]:
print(training_data.shape)

(42000, 784)


In [0]:
from sklearn.preprocessing import StandardScaler
standardized_data = StandardScaler().fit_transform(training_data)
print(standardized_data.shape)

(42000, 784)


In [0]:
covar_matrix = np.matmul(standardized_data.T , standardized_data)

print ( "The shape of variance matrix = ", covar_matrix.shape)

The shape of variance matrix =  (784, 784)


In [0]:
training_data

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


###Decision Tree on Original Data

In [0]:
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(training_data, labels, test_size=0.25, random_state=49)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.8514285714285714

###MDS

In [0]:
from sklearn.manifold import MDS
'''
embedding = MDS(n_components=16)
X_transformed = embedding.fit_transform(training_data[:5000])
'''

'\nembedding = MDS(n_components=16)\nX_transformed = embedding.fit_transform(training_data[:5000])\n'

In [0]:
X_transformed.shape

NameError: ignored

###Decision T on MDS Data

In [0]:
from prettytable import PrettyTable
t_mds_10k = PrettyTable(['DT_score_onMDSData', 'Dimensions', 'MDS_time','DecisionTree_time' ,'total_time', 'trust_mds','cont_mds' ,'lcmc_mds'])


def MDS_Decision_Tree (training_data, components,size):
  #Reducing to mentioned components
  #print("starting mds")
  mds_start_time = time.time()
  embedding = MDS(n_components=components)
  training_data_mds = embedding.fit_transform(training_data[:size])
  mds_time = time.time() - mds_start_time
  #print("ending mds")
  
  #Applying Decision Tree
  X_train, X_test, y_train, y_test = train_test_split(training_data_mds, labels[:size], test_size=0.25, random_state=49)
  clf_start_time = time.time()
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(X_train, y_train)
  clf_time = time.time() - clf_start_time
  score=clf.score(X_test,y_test)
  score = str(round(score, 4))

  total_time = mds_time+clf_time
  mds_time = str(round(mds_time, 4))
  clf_time = str(round(clf_time, 4))
  total_time = str(round(total_time, 4))


  trust_mds = 0
  cont_mds = 0
  lcmc_mds = 0
  
  Q = coranking.coranking_matrix(training_data[:size], training_data_mds)
  trust_mds = trustworthiness(Q, min_k=1, max_k=50).sum()/50
  cont_mds = continuity(Q, min_k=1, max_k=50).sum()/50
  lcmc_mds = LCMC(Q, min_k=1, max_k=50).sum()/50

  trust_mds = str(round(trust_mds, 4))
  cont_mds = str(round(cont_mds, 4))
  lcmc_mds = str(round(lcmc_mds, 4))
  
  #print(components)
  print("{}, {}, {}, {}, {}, {}, {}, {} ".format( score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds))
  print("*"*40)
  t_mds_10k.add_row([score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds])
  return

In [0]:
dimensions=784
size=10000
while dimensions >= 2:
  MDS_Decision_Tree (training_data, dimensions,size)
  dimensions=int(dimensions/2)

#print(t_mds_10k)

0.5856, 784, 6976.367, 9.8085, 6986.1755, 0.9774, 0.9735, 0.4517 
****************************************
0.5952, 392, 4906.8674, 4.739, 4911.6064, 0.9768, 0.9729, 0.4317 
****************************************
0.5912, 196, 3892.2962, 2.4492, 3894.7454, 0.9758, 0.9718, 0.4029 
****************************************
0.5788, 98, 3224.2941, 1.1935, 3225.4875, 0.9739, 0.9697, 0.3628 
****************************************
0.582, 49, 2879.376, 0.6225, 2879.9985, 0.9702, 0.9664, 0.3206 
****************************************
0.6072, 24, 2745.8403, 0.302, 2746.1423, 0.9646, 0.9626, 0.2927 
****************************************
0.6604, 12, 2683.8469, 0.1574, 2684.0044, 0.9562, 0.9584, 0.2882 
****************************************
0.6932, 6, 2659.2263, 0.0999, 2659.3261, 0.9459, 0.9625, 0.2855 
****************************************
0.184, 3, 219.7745, 0.0718, 219.8463, 0.5788, 0.5437, 0.005 
****************************************


In [0]:
print(t_mds_10k)

+--------------------+------------+-----------+-------------------+------------+-----------+----------+----------+
| DT_score_onMDSData | Dimensions |  MDS_time | DecisionTree_time | total_time | trust_mds | cont_mds | lcmc_mds |
+--------------------+------------+-----------+-------------------+------------+-----------+----------+----------+
|       0.5856       |    784     |  6976.367 |       9.8085      | 6986.1755  |   0.9774  |  0.9735  |  0.4517  |
|       0.5952       |    392     | 4906.8674 |       4.739       | 4911.6064  |   0.9768  |  0.9729  |  0.4317  |
|       0.5912       |    196     | 3892.2962 |       2.4492      | 3894.7454  |   0.9758  |  0.9718  |  0.4029  |
|       0.5788       |     98     | 3224.2941 |       1.1935      | 3225.4875  |   0.9739  |  0.9697  |  0.3628  |
|       0.582        |     49     |  2879.376 |       0.6225      | 2879.9985  |   0.9702  |  0.9664  |  0.3206  |
|       0.6072       |     24     | 2745.8403 |       0.302       | 2746.1423  |

In [0]:
print(time.time())

1575121651.2272756
